In [1]:
import pandas as pd
import numpy as np
import os
import scanpy as sc
import harmony
%matplotlib inline
#import bbknn
#import mnnpy
import seaborn as sns

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [4]:
import scipy.io
mtx = scipy.io.mmread('data/countsMatrix.mtx')

In [9]:
mtx = mtx.todense()

In [15]:
genes = pd.read_csv('data/genes.csv', header = None)[0]

In [25]:
cell = pd.read_csv('data/meta.csv').cell

In [26]:
meta = pd.read_csv('data/meta.csv', index_col = 0)

In [27]:
meta

,sample,DevelopmentalStage,DaysPostAmputation,cluster,X,Y,CellCyclePhase
cell,,,,,,,
AAACCTGAGCTAGTTC.1,SIGAB5,st40,3,Erythrocyte 4,-6.395430,0.832323,G2M
AAACCTGGTGGGTCAA.1,SIGAB5,st40,3,Myeloid 1,-2.428271,13.826715,G1
AAACCTGGTTTGTTGG.1,SIGAB5,st40,3,Beta ionocyte,-1.398049,-14.653897,G1
AAACGGGGTCGGCATC.1,SIGAB5,st40,3,Erythrocyte 4,-5.863765,0.489598,G2M
AAACGGGTCCTACAGA.1,SIGAB5,st40,3,Goblet cell,2.106444,-6.594430,G1
...,...,...,...,...,...,...,...
TTCTTAGAGTACCGGA.1,SIGAB10,st40,3,Erythrocyte 4,-5.879582,0.924197,G2M
TTGACTTAGAGTAAGG.1,SIGAB10,st40,3,Goblet cell,1.252149,-8.870151,G1
TTGCGTCTCAAGAAGT.1,SIGAB10,st40,3,Goblet cell,0.693335,-9.168521,G1


In [66]:
counts = pd.DataFrame(data = mtx,index = genes, columns = cell ).T

In [67]:
df = pd.read_csv('/work/Xenopus/Notebooks/V3/misc_files/XENLA_XenBase20190115_prot.txt', sep = '\t', index_col = 0)
df.index = [str(x).lower() for x in df.index]
mapper = {x:h for x,h in zip(df.index, df['Unnamed: 5'])}
df_map = pd.DataFrame({'Data_name':counts.columns,
              'lower_name': [x.lower() for x in counts.columns]
             })
df_map['Human_name'] = df_map.lower_name.map(mapper)
mapper = {x:h for x, h in zip(df_map['Data_name'], df_map['Human_name'])}

In [68]:
norm_df = harmony.utils.normalize_counts(counts)

In [69]:

norm_df.columns = [mapper[x] for x in norm_df.columns]
counts = norm_df.T
counts = counts[counts.index!= 'NoName']
counts = counts[~counts.index.isna()]
counts = counts.reset_index().drop_duplicates(keep = 'first', subset='index').set_index('index')


In [72]:
counts.to_csv('Aztekin_counts.tsv', sep = '\t')

In [74]:
meta['Dataset'] = 'Aztekin'
meta['Cell-type'] = meta['cluster']
meta['Species'] = 'Xenopus L'

In [76]:
meta.to_csv('Aztekin_meta.tsv', sep = '\t')

In [77]:
counts

cell,AAACCTGAGCTAGTTC.1,AAACCTGGTGGGTCAA.1,AAACCTGGTTTGTTGG.1,AAACGGGGTCGGCATC.1,AAACGGGTCCTACAGA.1,AAAGATGAGTACTTGC.1,AAAGATGCAAGTTAAG.1,AAAGATGGTGTTTGTG.1,AAAGATGTCCTCATTA.1,AAAGCAAAGTGGAGAA.1,...,TTAGTTCAGCACGCCT.1,TTCTACACAGCCACCA.1,TTCTCAAAGAAACCAT.1,TTCTCAATCCAGGGCT.1,TTCTCCTAGGCATTGG.1,TTCTTAGAGTACCGGA.1,TTGACTTAGAGTAAGG.1,TTGCGTCTCAAGAAGT.1,TTGTAGGCAGTACACT.1,TTTGCGCAGCGTGAAC.1.1
index,,,,,,,,,,,,,,,,,,,,,
TM6SF2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
LBP,0.0,1.107542,0.000000,0.0,0.0,1.130199,0.000000,0.000000,3.227889,0.000000,...,0.0,0.0,0.423155,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
ARHGAP28,0.0,0.000000,1.994018,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
TMEM200C,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
BCAP31,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.396495,0.902772,0.000000,0.000000,...,0.0,0.0,0.846310,0.0,2.020202,0.0,1.582529,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TBC1D10B,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
SRRM2,0.0,1.107542,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.488544,...,0.0,0.0,0.423155,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
GBP1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
